# Make VO table of extreme starbursts

Reprocessing the manually made csv file from https://docs.google.com/spreadsheets/d/1hF9ls8AMzzY--2I5tukqBmJf6xf7LO8lB4NTr3sFu08/edit#gid=0

In [1]:
from astropy.table import Table, Column
from astropy.coordinates import SkyCoord

import numpy as np

import yaml

from pymoc import MOC

from herschelhelp_internal.utils import coords_to_hpidx,  gen_help_id, inMoc

In [2]:
giants = Table.read('./data/Known_Monsters_Sheet1.csv')

In [3]:
giants

Name,RA,Dec,Redshift,Redshift source,Lbol/Lsun intrinsic,Lensed?,Original survey,Reference,Notes
str24,str22,str13,str10,str14,str11,str9,str17,str106,str152
9io9,02h09m41.3s,00d15m58.6s,2.553,"CO, H-alpha",1.00E+13,Y mu~10,Planck/ACT/VICS82,http://adsabs.harvard.edu/abs/2015MNRAS.452..502G,--
WISE J132934.18+224327.3,13h29m34.18s,22d43m27.3s,2.0439,H-alpha,1.30E+13,Y mu~11,WISE/Planck,http://adsabs.harvard.edu/abs/2017ApJ...843L..22D,--
Orochi,02h18m30.7s,-05d31m31s,3.4,CO,~1.30e13,Y mu~5,AzTEC-ASTE,http://adsabs.harvard.edu/abs/2011MNRAS.415.3081I,--
MMJ154506.4−344318,15h45m06.35s,-34d43m18.2s,04-May,est,>1e13,?,AzTEC-ASTE,http://adsabs.harvard.edu/abs/2015ApJ...808..121T,--
MMJ154132.7−350320,15h41m32.71s,-35d03m23.3s,3,est,13.7-13.9,Y mu~1.2,AzTEC-ASTE,http://adsabs.harvard.edu/abs/2015ApJ...808..121T,--
GN20,12h37m12s,62d22m12.100s,4.06,CO,2.90E+13,N,GOODS-N,http://adsabs.harvard.edu/abs/2009ApJ...694.1517D,--
SPT0103-45,15.797917,-45.648306,3.0917,ALMA 3mm CO,1.67E+13,mu = 5.34,SPT-SZ,https://ui.adsabs.harvard.edu/#abs/2016ApJ...826..112S/abstract,Contact: jspilker@utexas.edu
SPT0125-47,21.2795,-47.398889,2.5148,ALMA 3mm CO,3.33E+13,mu = 5.47,SPT-SZ,https://ui.adsabs.harvard.edu/#abs/2016ApJ...826..112S/abstract,Contact: jspilker@utexas.edu
SPT0243-49,40.786708,-46.35675,5.699,ALMA 3mm CO,1.63E+13,mu = 5.09,SPT-SZ,https://ui.adsabs.harvard.edu/#abs/2016ApJ...826..112S/abstract,Contact: jspilker@utexas.edu


## 1. Insist on RA Dec
We will also convert all RA Dec to decimal degrees

In [4]:
giants['RA'].fill_value = ''
giants['Dec'].fill_value = ''
giants = giants.filled()
giants = giants[(giants['RA'] != '' ) & (giants['Dec'] != '')]

In [5]:
for gal in giants:
    if 'h' in gal['RA']:
        print(gal['RA'])
        hours = gal['RA'].split('h')[0]
        minutes = gal['RA'].split('h')[1].split('m')[0]
        seconds = gal['RA'].split('h')[1].split('m')[1].split('s')[0]
        coords = SkyCoord(gal['RA'].replace(' ', ''), gal['Dec'].replace(' ', ''))
        print(coords.ra.value, coords.dec.value)
        gal['RA'] = str(coords.ra.value)
        gal['Dec'] = str(coords.dec.value)
        
    gal['Lensed?'] = gal['Lensed?'].replace('m', '')
    gal['Lensed?'] = gal['Lensed?'].replace('u', '')
    gal['Lensed?'] = gal['Lensed?'].replace(' ', '')
    gal['Lensed?'] = gal['Lensed?'].replace('N', '')
    gal['Lensed?'] = gal['Lensed?'].replace('Y', '')
    gal['Lensed?'] = gal['Lensed?'].replace('~', '')
    gal['Lensed?'] = gal['Lensed?'].replace('?', '')
    gal['Lensed?'] = gal['Lensed?'].replace('=', '')
    gal['Lensed?'] = gal['Lensed?'].replace('A', '')
    gal['Lensed?'] = gal['Lensed?'].replace('/', '')
    gal['Lensed?'] = gal['Lensed?'].replace('nlensed', '')
        
    

02h09m41.3s
32.42208333333333 0.2662777777777778
13h29m34.18s
202.39241666666663 22.724249999999998
02h18m30.7s
34.62791666666666 -5.525277777777777
15h45m06.35s
236.2764583333333 -34.721722222222226
15h41m32.71s
235.38629166666664 -35.05647222222222
12h37m12s
189.29999999999998 62.37002777777778


/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/row.py:54: StringTruncateWarning: truncated right side string(s) longer than 13 character(s) during assignment
  self._table.columns[item][self._index] = val
/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/row.py:54: StringTruncateWarning: truncated right side string(s) longer than 13 character(s) during assignment
  self._table.columns[item][self._index] = val
/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/row.py:54: StringTruncateWarning: truncated right side string(s) longer than 13 character(s) during assignment
  self._table.columns[item][self._index] = val
/Users/rs548/anaconda/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/row.py:54: StringTruncateWarning: truncated right side string(s) longer than 13 character(s) during assignment
  self._table.columns[item][self._index] = val
/Use

In [7]:
giants['RA'] = giants['RA'].astype(float)
giants['Dec'] = giants['Dec'].astype(float)

giants['Lbol/Lsun intrinsic'] = giants['Lbol/Lsun intrinsic'].astype('<U22')
for gal in giants:
    try:
        test = float(gal['Redshift'])
    except:
        gal['Redshift'] = '-99'
    gal['Lbol/Lsun intrinsic'] = gal['Lbol/Lsun intrinsic'].replace('~', '')
    gal['Lbol/Lsun intrinsic'] = gal['Lbol/Lsun intrinsic'].replace('>', '')
    gal['Lbol/Lsun intrinsic'] = gal['Lbol/Lsun intrinsic'].split('-')[0]
    if 'logSFR' in gal['Lbol/Lsun intrinsic']:
        gal['Lbol/Lsun intrinsic'] = '-99'
    elif not (('e' in gal['Lbol/Lsun intrinsic']) 
            or ('E' in gal['Lbol/Lsun intrinsic'])):
        gal['Lbol/Lsun intrinsic'] = str(10** float(gal['Lbol/Lsun intrinsic']) )

giants['Redshift'] = giants['Redshift'].astype(float)
giants['Redshift'][giants['Redshift'] < 0] =np.nan
giants['Lbol/Lsun intrinsic'] = giants['Lbol/Lsun intrinsic'].astype(float)
giants['Lbol/Lsun intrinsic'][giants['Lbol/Lsun intrinsic'] < 0] =np.nan

giants['Lensed?'][giants['Lensed?'] == ''] = '-99'
giants['Lensed?'] = giants['Lensed?'].astype(float)
giants['Lensed?'][giants['Lensed?'] < 0] = np.nan

In [9]:
giants['RA'].name = 'ra'
giants['Dec'].name = 'dec'
giants['Redshift'].name = 'redshift'
giants['Lbol/Lsun intrinsic'].name = 'lbol_intrinsic'
giants['Lensed?'].name = 'mu_lens'

In [10]:
giants.add_column(Column(gen_help_id(giants['ra'], giants['dec']),
                                   name="help_id"))
giants.add_column(Column(np.full(len(giants), "", dtype='<U18'),
                                   name="field"))

giants.add_column(Column(
    data=coords_to_hpidx(giants['ra'], giants['dec'], order=13),
    name="hp_idx"
))

In [11]:
giants['help_id'] = [s.replace('HELP', 'HELP_rare') for s in giants['help_id']]

In [12]:
fields_info = yaml.load(open("../../dmu2/meta_main.yml", 'r'))

for field in fields_info['fields']:

    field_moc = MOC(filename='../../dmu2/dmu2_field_coverages/{}_MOC.fits'.format(field['name']))
    giants['field'][inMoc(giants['ra'], giants['dec'], field_moc)] = field['name']
    


In [17]:
giants['help_id', 
        'hp_idx',
        'field', 
        'ra', 
        'dec', 
        'redshift', 
        'lbol_intrinsic',
        'mu_lens'][:10].show_in_notebook()

idx,help_id,hp_idx,field,ra,dec,redshift,lbol_intrinsic,mu_lens
0,HELP_rare_J020941.300+001558.600,291882535,Herschel-Stripe-82,32.42208333333333,0.26627777777,2.553,10000000000000.0,10.0
1,HELP_rare_J132934.180+224327.300,167978479,,202.39241666666663,22.7242499999,2.0439,13000000000000.0,11.0
2,HELP_rare_J021830.700-053131.000,289810432,XMM-LSS,34.62791666666666,-5.5252777777,3.4,13000000000000.0,5.0
3,HELP_rare_J154506.350-344318.200,700199708,,236.2764583333333,-34.721722222,nan,10000000000000.0,nan
4,HELP_rare_J154132.710-350323.300,700044386,,235.38629166666664,-35.056472222,3.0,50118723362727.15,1.2
5,HELP_rare_J123712.000+622212.100,184354581,HDF-N,189.29999999999998,62.3700277777,4.06,29000000000000.0,nan
6,HELP_rare_J010311.500-453853.902,579939617,,15.797917,-45.648306,3.0917,16700000000000.0,5.34
7,HELP_rare_J012507.080-472356.000,574017511,,21.2795,-47.398889,2.5148,33300000000000.0,5.47
8,HELP_rare_J024308.810-462124.300,553364920,,40.786708,-46.35675,5.699,16300000000000.0,5.09
9,HELP_rare_J030004.370-462124.300,553435566,,45.018208,-46.35675,3.5954,12600000000000.0,6.3


In [16]:
giants['help_id', 
        'hp_idx',
        'field', 
        'ra', 
        'dec', 
        'redshift', 
        'lbol_intrinsic',
        'mu_lens'].write('./data/rare_objects.csv', overwrite=True)